# Reference Run Ranking Performance Testing & Development Notebook

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

import sys
sys.path.append("./src/")
import refrunrank as rrr

import dqmexplore as dqme
from cmsdials.filters import OMSFilter, OMSPage

dials = dqme.utils.setupdials.setup_dials_object_deviceauth()

## CertHelper

In [ ]:
import importlib
importlib.reload(rrr.certhelper)

In [ ]:
json_dir = "/afs/cern.ch/user/r/rcruzcan/rcruzcan/SWAN_projects/RefRunRank/RefRunRank/jsons"
ch_refrunjson = "ch_refrunjson.json"
rr_goldenjson = 'Express-Collisions-2022-2023-2024_pixel-strip-track-good.json'

chdata = rrr.certhelper.CHRunData(
    os.path.join(json_dir, ch_refrunjson),
    os.path.join(json_dir, rr_goldenjson)
)

chdata.getGoodRuns()

In [ ]:
filters = {
    # "run_number":[(352564, 355103), 365889],
    # "reference_run_number": [352425],
    # "run_reconstruction_type": "express",
    # "reference_run_reconstruction_type": "express",
    "dataset": "/Express/Collisions2022/*"
}

chdata.applyFilter(filters=filters)

## OMS

In [ ]:
import importlib
importlib.reload(rrr.omsdata)

In [ ]:
runnbs = chdata.applyFilter(filters=filters)["run_number"].to_list()
omsdata = rrr.omsdata.OMSData(dials)

# Set the runs (i.e. filters to specify runs) that we will get data for 
omsdata.setRuns(runnbs[:100]) 
# Downloads the requested data
omsdata.fetchData("runs")
omsdata.fetchData("lumisections")

## Ranking

In [ ]:
import importlib
importlib.reload(rrr.ranking)

| Run Features                    | LS Features                              | Stats  |
|---------------------------------|------------------------------------------|--------|
| `b_field`                      | `delivered_lumi_per_lumisection`         | `mean` |
| `l1_triggers_counter`          | `recorded_lumi_per_lumisection`          | `std`  |
| `hlt_physics_throughput`       | `init_lumi`                              | `min`  |
| `init_lumi`                    | `recorded_lumi`                          | `25%`  |
| `crossing_angle_stop`          | `end_lumi`                               | `50%`  |
| `energy`                       | `pileup`                                 | `75%`  |
| `end_lumi`                     | `delivered_lumi`                         | `max`  |
| `hlt_physics_rate`             | `prescale_index`                         |        |
| `fill_number`                  |                                          |        |
| `crossing_angle_start`         |                                          |        |
| `beta_star_start`              |                                          |        |
| `initial_prescale_index`       |                                          |        |
| `hlt_physics_size`             |                                          |        |
| `l1_rate`                      |                                          |        |
| `run_number`                   |                                          |        |
| `beta_star_stop`               |                                          |        |
| `hlt_physics_counter`          |                                          |        |

<!-- Run features:
* `b_field`
* `l1_triggers_counter`
* `hlt_physics_throughput`
* `init_lumi`
* `crossing_angle_stop`
* `energy`
* `end_lumi`
* `hlt_physics_rate`
* `fill_number`
* `crossing_angle_start`
* `beta_star_start`
* `initial_prescale_index`
* `hlt_physics_size`
* `l1_rate`
* `run_number`
* `beta_star_stop`
* `hlt_physics_counter`

LS features:
* `delivered_lumi_per_lumisection`
* `recorded_lumi_per_lumisection`
* `init_lumi`
* `recorded_lumi`
* `end_lumi`
* `pileup`
* `delivered_lumi`
* `prescale_index`

Stats:
* `mean`
* `std`
* `min`
* `25%`
* `50%`
* `75%`
* `max` -->

In [ ]:
ranker = rrr.ranking.RunRanker(omsdata)

target = 356075
ftrs_dict = {
    "runs": [
        "energy",
        "hlt_physics_rate",
        "fill_number",
        "run_number",
        "hlt_physics_counter",
    ],
    "lumisections": {
        "mean":["init_lumi", "recorded_lumi_per_lumisection", "pileup"],
        "std": ["init_lumi", "recorded_lumi_per_lumisection", "pileup"],
        "min": ["init_lumi", "recorded_lumi_per_lumisection", "pileup"],
        "max": ["init_lumi", "recorded_lumi_per_lumisection", "pileup"],
        "50%": ["init_lumi", "recorded_lumi_per_lumisection", "pileup"],
    }
}

ranker.setFeatures(ftrs_dict)
ranker.constructFeatures()
ranker.ftrsDF

In [ ]:
rslts, wghts = ranker.refrank_pca(target, n_components=2)
print(wghts)
rslts